In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import sys
sys.path.append('../py_model')
from utils import init_logging
import logging 
import os

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# auto-encoder network architecture

In [2]:
def autoencoder(dimensions=[278, 1000, 100, 278]):
    """
    Build a stacked deep autoencoder with tied weights, that is w = wT.

    return a dict.
    
    Parameters
    ----------
    dimensions : list, optional
        The number of neurons for each layer of the autoencoder.
    Returns
    -------
    x : Tensor
        Input placeholder to the network
    z : Tensor
        Inner-most latent representation
    y : Tensor
        Output reconstruction of the input
    loss : Tensor
        Overall cost to use for training
    """    
    logging.info('activation : {}'.format('relu'))
    # input to the network
    x = tf.placeholder(tf.float32, [None, dimensions[0]], name='x')
    current_input = x
    logging.info('current_input : {}'.format(current_input.shape))
    #---------------------------
    # Build the encoder
    #---------------------------
    encoder = [] # for putting the weight of encoder, w1,w2,..
    for layer_i, n_output in enumerate(dimensions[1:]):
        logging.info('layer_i-encoder : {}'.format(layer_i))
        logging.info('n_output-encoder: {}'.format(n_output))
        n_input = int(current_input.get_shape()[1]) # [0]: batch_szie, [1]:input_dim
        logging.info('n_input : {}'.format(n_input))
        W = tf.Variable(
            tf.random_uniform([n_input, n_output],
                              minval = -1.0 / math.sqrt(n_input),
                              maxval = 1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        # saving layer of encoding for decoder
        encoder.append(W)
        output = tf.nn.relu(tf.matmul(current_input, W) + b)
        # assign current_input
        current_input = output
    #---------------------------
    # latent representation (output of encoder)
    #---------------------------
    z = current_input
    encoder.reverse() # [...,w2,w1]
    
    #---------------------------
    # Build the decoder using the same weights
    #---------------------------
    for layer_i, n_output in enumerate(dimensions[:-1][::-1]):
        logging.info('layer_i-decoder : {}'.format(layer_i))
        logging.info('n_output-decoder : {}'.format(n_output))
        W = tf.transpose(encoder[layer_i])
        b = tf.Variable(tf.zeros([n_output]))
        output = tf.nn.relu(tf.matmul(current_input, W) + b)
        # assign current_input
        current_input = output

    # now have the reconstruction through the network
    y = current_input
    # Define loss and, minimize the mean squared error
    loss = tf.reduce_mean(tf.pow(x - y, 2)) 
    return {'x': x, 'z': z, 'y': y, 'loss': loss}


# input

In [12]:
log_dir = '../log/auto_encoder'
checkpoint_dir = '../log/auto_encoder/checkpoints'
prediction_dir = '../log/auto_encoder/predictions'
init_logging(log_dir)

In [4]:
pad_zero = True
# reload again for filling
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_comoress = ['index','TARGET', 'SK_ID_CURR']
df.drop(no_need_to_comoress, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
print (df.shape)
if pad_zero == True:
    # preprocessing for feature scailing
    df.replace(np.nan, 0, inplace = True)
    logging.info('pad_zero : {}'.format('True'))
else:
    logging.info('pad_zero : {}'.format('False'))
# preprocessing for feature scailing ignoring nan
for f in df.columns.tolist():
    mean = df[f].mean()
    std = df[f].std()
    df[f] = (df[f] - mean) / std
logging.info('input of ae : {}'.format(df.shape))

(356251, 281)
(356251, 278)


pad_zero : True
input of ae : (356251, 278)


# raw_feature_generator
### raw_feature_generator which allows to quickly set up Python generators that can automatically turn image files on disk into batches of pre-processed tensors. This is what we will use here.

In [5]:
length = len(df)
print ('length',length)
idx = np.arange(length) # 1-D array
print ('idx', idx.shape, idx)


length 356251
idx (356251,) [     0      1      2 ... 356248 356249 356250]


In [6]:
# def raw_feature_generator(batch_size = 128, shuffle = True, num_epochs = 10000, allow_smaller_final_batch = False):
#     epoch_num = 0
#     while epoch_num < num_epochs:
#         if shuffle:
#             np.random.shuffle(idx)
#         for i in range(0, length, batch_size):
#             batch_idx = idx[i: i + batch_size]
#             if not allow_smaller_final_batch and len(batch_idx) != batch_size:
#                 break # terminate the loop
#             yield df.values[batch_idx]
#         epoch_num += 1
    

In [10]:
def raw_feature_generator(batch_size = 128, shuffle = True, allow_smaller_final_batch = False):
    if shuffle:
        np.random.shuffle(idx)
    for i in range(0, length, batch_size):
        batch_idx = idx[i: i + batch_size]
        if not allow_smaller_final_batch and len(batch_idx) != batch_size:
            break # terminate the loop
        yield df.values[batch_idx]
        
def save(step, averaged = False):
    '''
    save the model
    '''
    #--------
    # create saver object
    #--------
    if averaged:
        saver = tf.train.Saver(self.ema.variables_to_restore(), max_to_keep=1)
        checkpoint_dir_averaged = checkpoint_dir + '_avg'
    else:
        saver = tf.train.Saver(max_to_keep=1)
    
    if not os.path.isdir(checkpoint_dir):
        logging.info('creating checkpoint directory {}'.format(checkpoint_dir))
        os.mkdir(checkpoint_dir)

    model_path = os.path.join(checkpoint_dir, 'model')
    logging.info('saving model to {}'.format(model_path))
    saver.save(sess, model_path)
    
def restore(step = None, averaged = False):
    '''
    restore the model.
    
    paras:
    -------------
    step: 
    '''
    if averaged:
        saver = tf.train.Saver(self.ema.variables_to_restore(), max_to_keep=1)
        checkpoint_dir_averaged = checkpoint_dir + '_avg'
    else:
        saver = tf.train.Saver(max_to_keep=1)
    #-------
    # there are two way to use restore function based on if step is given.
    #--------
    if not step:
        model_path = tf.train.latest_checkpoint(checkpoint_dir)
        logging.info('restoring model parameters from {}'.format(model_path))
        saver.restore(sess, model_path)
    else:
        model_path = os.path.join(
            checkpoint_dir, 'model{}-{}'.format('_avg' if averaged else '', step)
        )
        logging.info('restoring model from {}'.format(model_path))
        saver.restore(sess, model_path)
        
def test_batch_generator(batch_size):
    return self.batch_generator(
        batch_size=batch_size,
        df= self.test_df,
        shuffle=False,
        num_epochs=1,
        is_test=True
    )


def batch_generator(batch_size, df, shuffle=True, num_epochs=10000, is_test=False):
    batch_gen = df.batch_generator(batch_size, shuffle=shuffle, num_epochs=num_epochs, allow_smaller_final_batch=is_test)
    for batch in batch_gen:
        batch['order_dow_history'] = np.roll(batch['order_dow_history'], -1, axis=1)
        batch['order_hour_history'] = np.roll(batch['order_hour_history'], -1, axis=1)
        batch['days_since_prior_order_history'] = np.roll(batch['days_since_prior_order_history'], -1, axis=1)
        batch['order_number_history'] = np.roll(batch['order_number_history'], -1, axis=1)
        batch['next_is_ordered'] = np.roll(batch['is_ordered_history'], -1, axis=1)
        batch['is_none'] = batch['product_id'] == 0
        if not is_test:
            batch['history_length'] = batch['history_length'] - 1
        yield batch     
        
def predict(chunk_size = 2048, prediction_dir):
    
    if not os.path.isdir(prediction_dir):
        os.makedirs(prediction_dir)


    test_generator = test_batch_generator(chunk_size)
    for i, test_batch_df in enumerate(test_generator):
        if i % 100 == 0:
            print i*chunk_size

        test_feed_dict = {
            getattr(self, placeholder_name, None): data
            for placeholder_name, data in test_batch_df if hasattr(self, placeholder_name)
        }
        if hasattr(self, 'keep_prob'):
            test_feed_dict.update({self.keep_prob: 1.0})
        if hasattr(self, 'is_training'):
            test_feed_dict.update({self.is_training: False})

        tensor_names, tf_tensors = zip(*self.prediction_tensors.items())
        np_tensors = self.session.run(
            fetches=tf_tensors,
            feed_dict=test_feed_dict
        )
        for tensor_name, tensor in zip(tensor_names, np_tensors):
            prediction_dict[tensor_name].append(tensor)

    for tensor_name, tensor in prediction_dict.items():
        np_tensor = np.concatenate(tensor, 0)
        save_file = os.path.join(self.prediction_dir, '{}.npy'.format(tensor_name))
        logging.info('saving {} with shape {} to {}'.format(tensor_name, np_tensor.shape, save_file))
        # save
        np.save(save_file, np_tensor)

 

# training 

In [11]:
# display_step
log_interval = 100
# learning_rate
learning_rate = 0.001
# define auto-encoder network architecture
ae = autoencoder(dimensions=[278, 140, 70, 30])
# optimizer
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(ae['loss'])

# Initialize the variables (i.e. assign their default value)
# We create a session to use the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Training
batch_size = 256
n_epochs = 10
step = 0
min_steps_to_checkpoint = 100
enable_parameter_averaging = False
logging.info('learning_rate : {}'.format(learning_rate))
logging.info('batch_size : {}'.format(batch_size))
logging.info('n_epochs : {}'.format(n_epochs))
logging.info('log_interval : {}'.format(log_interval))
logging.info('min_steps_to_checkpoint : {}'.format(min_steps_to_checkpoint))
logging.info('enable_parameter_averaging : {}'.format(enable_parameter_averaging))

#-------------------------
# training - fit
#-------------------------
for epoch in range(n_epochs):
    epoch_cost = 0.0
    for batch_xs in raw_feature_generator(batch_size = batch_size):
        _, batch_loss = sess.run([optimizer, ae['loss']], feed_dict={ae['x']: batch_xs})
        epoch_cost += batch_loss / batch_size
        step += 1
    metric_log = (
                "[[epoch {:>8}]]     "
                "[[loss at this epoch]]     loss: {:<12}"
            ).format(epoch, round(epoch_cost, 8))
    logging.info(metric_log)
    if step > min_steps_to_checkpoint:
        print ('step : {}'.format(step))
        save(step)
        if enable_parameter_averaging:
            save(step, averaged=True)
restore()
predict(prediction_dir)

activation : relu
current_input : (?, 278)
layer_i-encoder : 0
n_output-encoder: 140
n_input : 278
layer_i-encoder : 1
n_output-encoder: 70
n_input : 140
layer_i-encoder : 2
n_output-encoder: 30
n_input : 70
layer_i-decoder : 0
n_output-decoder : 70
layer_i-decoder : 1
n_output-decoder : 140
layer_i-decoder : 2
n_output-decoder : 278
learning_rate : 0.001
batch_size : 256
n_epochs : 10
log_interval : 100
min_steps_to_checkpoint : 100
enable_parameter_averaging : False
[[epoch        0]]     [[loss at this epoch]]     loss: 3.96424152  
saving model to ../log/auto_encoder/checkpoints/model


step : 1391


[[epoch        1]]     [[loss at this epoch]]     loss: 3.11947044  
saving model to ../log/auto_encoder/checkpoints/model


step : 2782


[[epoch        2]]     [[loss at this epoch]]     loss: 2.8966567   
saving model to ../log/auto_encoder/checkpoints/model


...

[[epoch        9]]     [[loss at this epoch]]     loss: 2.43207278  
saving model to ../log/auto_encoder/checkpoints/model


step : 13910


restoring model parameters from ../log/auto_encoder/checkpoints/model


INFO:tensorflow:Restoring parameters from ../log/auto_encoder/checkpoints/model


Restoring parameters from ../log/auto_encoder/checkpoints/model
